## Bloco 04

> ### Validação cruzada


---

**Motivação**

Ao trabalhamos com um *dataset*, podemos dividir em treino e teste a partir de uma semente aleatória de forma que possamos realizar o cálculo com um conjunto que seja eficiente para uma determinada técnica, mas que não represente os dados como um todo. Dessa forma, podemos ter um vazamento de dados.

É importante, portanto, buscar encontrar uma *validação cruzada*, sendo que caso os resultados sejam consistentes para diferentes divisões da base de dados, a confiabilidade seja maior.

---

In [1]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random

In [2]:
df = pd.read_csv('data_artigo.csv')
df = df.drop("Unnamed: 0",axis=1)
df.head()

,Amostra,Sexo,Categoria Idade,Idade,MMP1,MMP3,MMP9,MMP11,MMP24,MMP27,BARD1,BCL7C,CLPTM1L,TERT,CDKN1A
0,GTEX-U8XE,male,Jovem,30-39,0.415301,0.019599,0.609227,0.088210,0.000014,0.851644,0.354668,0.472280,0.910613,0.456960,0.770863
1,GTEX-UPJH,male,Adulto,50-59,0.000005,0.007997,0.159422,0.038826,0.503206,0.168324,0.491779,0.604135,0.656658,0.020528,0.301729
2,GTEX-UPK5,male,Adulto,40-49,0.000000,0.007150,0.000003,0.047462,0.000003,0.310585,0.215032,0.536110,0.690726,0.000000,0.230042
3,GTEX-V1D1,male,Adulto,50-59,0.000000,0.000000,0.062979,0.000001,0.000034,0.523176,0.488843,0.430027,0.459522,0.183716,0.266740
4,GTEX-VJYA,male,Idoso,60-69,0.000000,0.019699,0.329350,0.037106,0.444207,0.744908,0.250793,0.411447,0.618432,0.223564,0.134728


In [3]:
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 2499

FEATURES = df.columns.values[4:]
TARGET = ["Categoria Idade"]

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

NUM_FOLDS = 3
NUM_CPU_CORES = 4

modelo = KNeighborsClassifier()

scores = cross_val_score(
    modelo,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="accuracy",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.66666667 0.60606061 0.51515152]

A média dos scores é de:  0.5959595959595959


In [5]:
scores = cross_val_score(
    modelo,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="accuracy",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.66666667 0.60606061 0.51515152]

A média dos scores é de:  0.5959595959595959


Uma vez que o método mais eficiente para o projeto foi o de classificação de K vizinhos, já que os dados de *target* são categóricos, usamos como método de pontuação a acurácia.

In [6]:
N_BEIGHBORS = 15
NUM_FOLDS = 3
NUM_CPU_CORES = 4

modelo = KNeighborsClassifier(
    n_neighbors=N_BEIGHBORS,
    n_jobs=NUM_CPU_CORES,
)

scores = cross_val_score(
    modelo,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="accuracy",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.66666667 0.66666667 0.60606061]

A média dos scores é de:  0.6464646464646465


In [9]:
from sklearn.metrics import accuracy_score

N_BEIGHBORS = 15
NUM_FOLDS = 3
NUM_CPU_CORES = 4

modelo = KNeighborsClassifier(
    n_neighbors=N_BEIGHBORS,
    n_jobs=NUM_CPU_CORES,
)

modelo.fit(X_treino, y_treino)

y_previsao = modelo.predict(X_teste)
accuracy = accuracy_score(y_teste, y_previsao)

print(f"A acurácia do modelo de K vizinhos do conjunto de teste foi de {accuracy}.")

A acurácia do modelo de K vizinhos do conjunto de teste foi de 0.8181818181818182.


---

**Aplicação de K vizinhos com Pipeline**

In [10]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

NUM_VIZINHOS = 3
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="accuracy",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.5        0.6        0.6        0.6        0.7        0.5
 0.5        0.4        0.6        0.44444444]

A média dos scores é de:  0.5444444444444445


In [11]:
NUM_VIZINHOS = 7
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="accuracy",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.6        0.7        0.7        0.7        0.7        0.7
 0.5        0.4        0.6        0.66666667]

A média dos scores é de:  0.6266666666666667


In [12]:
NUM_VIZINHOS = 7

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier(n_neighbors=NUM_VIZINHOS),
)
modelo_knn_composto.fit(X_treino, y_treino)

y_previsao = modelo_knn_composto.predict(X_teste)
accuracy = accuracy_score(y_teste, y_previsao)

print(f"A acurácia do modelo de K vizinhos do conjunto de teste foi de {accuracy}.")

A acurácia do modelo de K vizinhos do conjunto de teste foi de 0.8181818181818182.


---

Tendo em vista as análises, é possível concluir que a acurácia resultada no bloco 2 pode ser confirmada pela validação cruzada.